# Interacting with Big Query using pandas

A working example of using Google Big query with pandas

### Resources used

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_gbq.html

https://pandas-gbq.readthedocs.io/en/latest/howto/authentication.html

https://pypi.org/project/google-auth/

In [1]:
# load required packages

import os
import pandas as pd
from google.oauth2 import service_account
from dotenv import load_dotenv

In [2]:
# load values from .env to environment
load_dotenv()


service_account_json = os.getenv("SERVICE_ACCOUNT_JSON")
project_id = os.getenv("PROJECT_ID")

In [3]:
# create Google credentials from service account json file

credentials = service_account.Credentials.from_service_account_file(
    service_account_json,
)

In [4]:
# column names
my_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]

# load iris data
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", 
                   header = None,
                  names = my_columns)

print(data.shape)
data.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Write to Big Query

In [5]:
# write to Big Query
data.to_gbq("an_example_dataset.iris", 
            project_id=project_id,
           if_exists = "replace",
           credentials=credentials)

## Read from Big Query

In [6]:
# read from Big query

q = """
SELECT * FROM `example-project-bq-test.an_example_dataset.iris`
WHERE class='Iris-versicolor';
"""

out_data = pd.read_gbq(q, 
                       project_id=project_id,
                       credentials=credentials,
                       progress_bar_type=None)

/home/fuzzy/.local/share/virtualenvs/bq-write-play-dzBGmu5t/lib/python3.7/site-packages/pandas_gbq/gbq.py:555: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  progress_bar_type=progress_bar_type,


In [7]:
out_data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,2.5,3.0,1.1,Iris-versicolor
1,5.0,2.0,3.5,1.0,Iris-versicolor
2,5.7,2.6,3.5,1.0,Iris-versicolor
3,6.0,2.2,4.0,1.0,Iris-versicolor
4,5.8,2.6,4.0,1.2,Iris-versicolor


In [8]:
out_data.tail()

,sepal_length,sepal_width,petal_length,petal_width,class
45,6.7,3.1,4.7,1.5,Iris-versicolor
46,6.1,2.8,4.7,1.2,Iris-versicolor
47,7.0,3.2,4.7,1.4,Iris-versicolor
48,6.1,2.9,4.7,1.4,Iris-versicolor
49,6.3,3.3,4.7,1.6,Iris-versicolor


In [9]:
out_data.shape

(50, 5)